In [63]:
#AI-TECHGYM-1-8-A-3
#自然言語処理

from janome.tokenizer import Tokenizer

#形態素解析のオブジェクト
text = Tokenizer()

#txtファイルからデータの読み込み
text_file = open("techgym-AI.txt")
txt = text_file.read()
 
#読み込んだデータを形態素解析
lines = txt.split("\n")
for i in lines:
    print(i)
    print("\n")
    text_c = text.tokenize(i)    
    for j in text_c:
        print(j)
    print("\n")


月額2万円の「プログラミングライフ」


月額	名詞,一般,*,*,*,*,月額,ゲツガク,ゲツガク
2	名詞,数,*,*,*,*,2,*,*
万	名詞,数,*,*,*,*,万,マン,マン
円	名詞,接尾,助数詞,*,*,*,円,エン,エン
の	助詞,連体化,*,*,*,*,の,ノ,ノ
「	記号,括弧開,*,*,*,*,「,「,「
プログラミング	名詞,サ変接続,*,*,*,*,プログラミング,プログラミング,プログラミング
ライフ	名詞,一般,*,*,*,*,ライフ,ライフ,ライフ
」	記号,括弧閉,*,*,*,*,」,」,」


「テックジム（TechGYM）」は、どんなテクノロジーにも即座に対応できる「自走できるエンジニア」をゴールにした自習形式のプログラマー養成塾です。


「	記号,括弧開,*,*,*,*,「,「,「
テック	名詞,一般,*,*,*,*,テック,テック,テック
ジム	名詞,一般,*,*,*,*,ジム,ジム,ジム
（	記号,括弧開,*,*,*,*,（,（,（
TechGYM	名詞,固有名詞,組織,*,*,*,TechGYM,*,*
）	記号,括弧閉,*,*,*,*,）,）,）
」	記号,括弧閉,*,*,*,*,」,」,」
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、
どんな	連体詞,*,*,*,*,*,どんな,ドンナ,ドンナ
テクノロジー	名詞,一般,*,*,*,*,テクノロジー,テクノロジー,テクノロジー
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
も	助詞,係助詞,*,*,*,*,も,モ,モ
即座	名詞,一般,*,*,*,*,即座,ソクザ,ソクザ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
対応	名詞,サ変接続,*,*,*,*,対応,タイオウ,タイオー
できる	動詞,自立,*,*,一段,基本形,できる,デキル,デキル
「	記号,括弧開,*,*,*,*,「,「,「
自	接頭詞,名詞接続,*,*,*,*,自,ジ,ジ
走	動詞,自立,*,*,五段・ラ行,体言接続特殊２,走る,ハシ,ハシ
できる	動詞,自立,*,*,一段,基本形,できる,デキル,デキル
エンジニア	名詞,一般,*,*,*,*,エンジニア,エンジニア,エンジニア
」	記号,括弧閉,*,*,*,*,」,」,」
を	助詞,格助詞,一般

In [64]:
txt

'月額2万円の「プログラミングライフ」\n「テックジム（TechGYM）」は、どんなテクノロジーにも即座に対応できる「自走できるエンジニア」をゴールにした自習形式のプログラマー養成塾です。\nどんなレベルからでも始められますので、全くのプログラミング初心者からベテランエンジニアまで、各個人のスキルレベルにあわせて学ぶことができます。\nなお、最速でプログラミングスキルの獲得をしたい方にはテックジムのカリキュラムがオススメです。\n開催期間にあわせて、好きな時間に好きなだけ、何回でも参加することが可能で、課題を持ち帰ることも可能です。（提携校により開催頻度や開催時間が違います。）\n自習を見守るトレーナーは、経験豊富な現役プログラマーです。カリキュラム範囲以外でも、当人の得意分野であれば、技術的な相談相手になってくれますので、「卒業」という概念もありません。会員同士の交流は自由ですので、エンジニア仲間や生涯の仲間を見つける感覚でご参加ください。\nまさに、いつでも参加でき、ずっとスキルを磨き続けることのできる「ジム」の要素が「テックジム（TechGYM）」の大きな特徴です。\n入会金は3万円（現在は無料）、月額2万円の「プログラミングライフ」をお楽しみください。\nもっとも効率的なスキル習得「テックジム方式」とは？\n「テックジム方式」とは、基礎知識なしでも、座学なしでプログラミングに専念できるように設計されたプログラミングのカリキュラムメソッドです。\n優れたエンジニアの多くは、職業訓練学校や研修スクールで学ぶような学び方をしません。\n作りたいものを作っていたら、自然に習得できているのです。そして、優れた指導者（メンターや師匠）との出会いがエンジニアとしてのプロフェッショナル性を高めます。\nこの自発性とプロフェッショナル性を兼ね備えたのが「テックジム方式」です。\n授業を聞いたりテキストを読むことで失う時間を、純粋にプログラミング時間に向けることで、習得効率は飛躍的に向上し、モチベーションも維持できます。\nその時点で知らなくていい知識や概念を可能な限り削ぎ落とし、知るべきタイミングが訪れた時にはすでに課題を通じて覚えているようなスキル習得の自動化を図ります。\n講座で用いるサンプルソースや課題は、最も効率よく学べるように細部に渡って設計されています。\nまるで

In [65]:
lines

['月額2万円の「プログラミングライフ」',
 '「テックジム（TechGYM）」は、どんなテクノロジーにも即座に対応できる「自走できるエンジニア」をゴールにした自習形式のプログラマー養成塾です。',
 'どんなレベルからでも始められますので、全くのプログラミング初心者からベテランエンジニアまで、各個人のスキルレベルにあわせて学ぶことができます。',
 'なお、最速でプログラミングスキルの獲得をしたい方にはテックジムのカリキュラムがオススメです。',
 '開催期間にあわせて、好きな時間に好きなだけ、何回でも参加することが可能で、課題を持ち帰ることも可能です。（提携校により開催頻度や開催時間が違います。）',
 '自習を見守るトレーナーは、経験豊富な現役プログラマーです。カリキュラム範囲以外でも、当人の得意分野であれば、技術的な相談相手になってくれますので、「卒業」という概念もありません。会員同士の交流は自由ですので、エンジニア仲間や生涯の仲間を見つける感覚でご参加ください。',
 'まさに、いつでも参加でき、ずっとスキルを磨き続けることのできる「ジム」の要素が「テックジム（TechGYM）」の大きな特徴です。',
 '入会金は3万円（現在は無料）、月額2万円の「プログラミングライフ」をお楽しみください。',
 'もっとも効率的なスキル習得「テックジム方式」とは？',
 '「テックジム方式」とは、基礎知識なしでも、座学なしでプログラミングに専念できるように設計されたプログラミングのカリキュラムメソッドです。',
 '優れたエンジニアの多くは、職業訓練学校や研修スクールで学ぶような学び方をしません。',
 '作りたいものを作っていたら、自然に習得できているのです。そして、優れた指導者（メンターや師匠）との出会いがエンジニアとしてのプロフェッショナル性を高めます。',
 'この自発性とプロフェッショナル性を兼ね備えたのが「テックジム方式」です。',
 '授業を聞いたりテキストを読むことで失う時間を、純粋にプログラミング時間に向けることで、習得効率は飛躍的に向上し、モチベーションも維持できます。',
 'その時点で知らなくていい知識や概念を可能な限り削ぎ落とし、知るべきタイミングが訪れた時にはすでに課題を通じて覚えているようなスキル習得の自動化を図ります。',
 '講座で用

In [34]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [89]:
sc = StandardScaler()
X_std = sc.fit_transform(cancer.data[:,0].reshape(-1,1))
print(X_std[:5])

[[ 1.09706398]
 [ 1.82982061]
 [ 1.57988811]
 [-0.76890929]
 [ 1.75029663]]


In [90]:
sc = StandardScaler()
X_std = sc.fit_transform(cancer.data)
print(X_std[:5])

[[ 1.09706398e+00 -2.07333501e+00  1.26993369e+00  9.84374905e-01
   1.56846633e+00  3.28351467e+00  2.65287398e+00  2.53247522e+00
   2.21751501e+00  2.25574689e+00  2.48973393e+00 -5.65265059e-01
   2.83303087e+00  2.48757756e+00 -2.14001647e-01  1.31686157e+00
   7.24026158e-01  6.60819941e-01  1.14875667e+00  9.07083081e-01
   1.88668963e+00 -1.35929347e+00  2.30360062e+00  2.00123749e+00
   1.30768627e+00  2.61666502e+00  2.10952635e+00  2.29607613e+00
   2.75062224e+00  1.93701461e+00]
 [ 1.82982061e+00 -3.53632408e-01  1.68595471e+00  1.90870825e+00
  -8.26962447e-01 -4.87071673e-01 -2.38458552e-02  5.48144156e-01
   1.39236330e-03 -8.68652457e-01  4.99254601e-01 -8.76243603e-01
   2.63326966e-01  7.42401948e-01 -6.05350847e-01 -6.92926270e-01
  -4.40780058e-01  2.60162067e-01 -8.05450380e-01 -9.94437403e-02
   1.80592744e+00 -3.69203222e-01  1.53512599e+00  1.89048899e+00
  -3.75611957e-01 -4.30444219e-01 -1.46748968e-01  1.08708430e+00
  -2.43889668e-01  2.81189987e-01]
 [ 1.5

In [49]:
#AI-TECHGYM-1-4-A-3
#教師なし学習 k-mean法

# データ加工・処理・分析ライブラリ
import pandas as pd
from pandas import Series, DataFrame

# 可視化ライブラリ
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# k-means法を使うためのインポート
from sklearn.cluster import KMeans

# データがあるurl の指定
file_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv'

# データを取得して展開する
#対象データを読み込み
shoppers= pd.read_csv(file_url)

# データの列の絞り込み
shoppers_sub = shoppers[['Administrative_Duration','Informational_Duration','ProductRelated_Duration','Region','SpecialDay']]

# KMeansクラスの初期化
kmeans = KMeans(init='random', n_clusters=6, random_state=0)

# クラスターの重心を計算
kmeans.fit(shoppers_sub)

# クラスター番号をpandasのSeriesオブジェクトに変換
labels = pd.Series(kmeans.labels_, name='cluster_number')

# 金融機関のデータにクラスター番号のデータを結合
shoppers_with_cluster = pd.concat([shoppers, labels], axis=1)

# 分割のための区切りを設定
bins = [0,0.2,0.4,0.6,0.8,1]

# 上の区切りをもとに金融機関のデータを分割し、qcut_age変数に各データの年齢層を設定
qcut_sp = pd.cut(shoppers_with_cluster.SpecialDay, bins, right=False)

# クラスタ番号と年齢層を結合
df = pd.concat([shoppers_with_cluster.cluster_number, qcut_sp], axis=1)

# クラスタ番号と年齢層を軸に集計し、年齢層を列に設定
cross_cluster_sp = df.groupby(['cluster_number', 'SpecialDay']).size().unstack().fillna(0)
display(cross_cluster_sp)

# 分割のための区切りを設定
bins_2 = [1,2,3,4,5,6,7,8,9]

# 上の区切りをもとに金融機関のデータを分割し、qcut_age変数に各データの年齢層を設定
qcut_r = pd.cut(shoppers_with_cluster.Region, bins_2, right=False, labels=[1,2,3,4,5,6,7,8])

# クラスタ番号と年齢層を結合
df = pd.concat([shoppers_with_cluster.cluster_number, qcut_r], axis=1)

# クラスタ番号と年齢層を軸に集計し、年齢層を列に設定
cross_cluster_r = df.groupby(['cluster_number', 'Region']).size().unstack().fillna(0)
display(cross_cluster_r)


SpecialDay,"[0.0, 0.2)","[0.2, 0.4)","[0.4, 0.6)","[0.6, 0.8)","[0.8, 1.0)"
cluster_number,,,,,
0,386,6,7,2,15
1,10,0,0,0,0
2,1096,9,28,26,20
3,2807,49,48,83,71
4,6657,114,159,240,218
5,123,0,1,0,1


Region,1,2,3,4,5,6,7,8
cluster_number,,,,,,,,
0,181,39,83,35,12,25,17,15
1,8,0,1,1,0,0,0,0
2,464,119,239,110,27,64,78,39
3,1145,302,616,302,79,212,207,120
4,2912,667,1444,727,196,500,453,260
5,70,9,20,7,4,4,6,0
